In [10]:
import cv2
import pytesseract
from pynput.keyboard import Key, Controller
import time
from webbot import Browser 
import pyautogui
import tkinter as tk
import pyautogui
import random
import numpy as np
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

def set_up(text):
    length = len(text)
    num_chunks = 12
    start_points = random.choices(range(length), k=11)
    start_points.append(0)
    start_points.append(length)
    start_points.sort()
    weights = [0.176, 0.20799999999999996, 0.154, 0.132, 0.11, 0.088, 0.066, 0.044, 0.022]
    weights.reverse()
    speeds = random.choices(range(3,12), weights=weights, k=14)
    speeds = np.array(speeds)
    speeds = speeds/100
    return num_chunks, start_points, speeds

# Use instructions
1. Run the two cells above (importing packages & defining a useful function)
2. Run the cell below just as a race is about to start, it'll do the rest. If something fails, just exit the race and try again, incomplete races are not included in the final stats
2. Laugh at the world

In [38]:
# Fixing location, adding a "top" limit
if pyautogui.locateOnScreen('screenshots/top_of_the_text_box_v2.PNG'):
    top_limit = pyautogui.locateOnScreen('screenshots/top_of_the_text_box_v2.PNG')
elif pyautogui.locateOnScreen('screenshots/top_of_the_text_box_v3.PNG'):
    top_limit = pyautogui.locateOnScreen('screenshots/top_of_the_text_box_v3.PNG')
elif pyautogui.locateOnScreen('screenshots/top_of_the_text_box_v4.PNG'):
    top_limit = pyautogui.locateOnScreen('screenshots/top_of_the_text_box_v4.PNG')

# Finding typing box
if pyautogui.locateOnScreen('screenshots/box3.PNG'):
    typing_box = pyautogui.locateOnScreen('screenshots/box3.PNG')
elif pyautogui.locateOnScreen('screenshots/type_box_v2.PNG'):
    typing_box = pyautogui.locateOnScreen('screenshots/type_box_v2.PNG')
elif pyautogui.locateOnScreen('screenshots/type_box_v3.PNG'):
    typing_box = pyautogui.locateOnScreen('screenshots/type_box_v3.PNG')

# Calculate coordinates of screenshot
new_left = typing_box[0]
new_top = top_limit[1] + 10
new_width = typing_box[2]
new_height = typing_box[1] - top_limit[1] - 15
good_coords = [new_left, new_top, new_width, new_height]

# Take screenshot
image_text = pyautogui.screenshot(region=good_coords)
image_text.save(r'screenshots/good_coords.png')

# Read the text from the image and replace weird characters
img = cv2.imread('screenshots/good_coords.png')
text = pytesseract.image_to_string(img)
text = text.replace('\n', ' ')
text = text.replace('|', 'I')
text = text.replace('’', '\'')
text = text.replace('‘', '')
text = text.replace('\“', '\"')
text = text.replace('[', 'I')

# Click on type racer's typing box
text_box_center = pyautogui.center(typing_box)
coordx, coordy = text_box_center
pyautogui.click(coordx, coordy)

# Divide text in parts, return # of chunks and all start & end points
num_chunks, start_points, speeds = set_up(text)

# Initialize keyboard simulators
keyboard = Controller()

# Override speeds
speeds = [0.05, 0.04, 0.03, 0.07, 0.06, 0.05, 0.04, 0.03, 0.05, 0.1 , 0.07, 0.03, 0.04, 0.05]

# Iterate through each of the chunks to type stuff at slightly different speeds
for counter, item in enumerate(start_points):
    # Choose a random speed
    random_speed = speeds[counter]
    
    # Type each of the chunks of text at chosen speed
    if counter + 1 < len(start_points):
        start = int(item)
        end = start_points[counter + 1]
        to_be_typed = text[start:end]
        for char in to_be_typed:
            keyboard.press(char)
            keyboard.release(char)
            time.sleep(random_speed)